In [42]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import random
import json
import re

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# Helper Functions

In [43]:
def viewOneTrip(df,tripID):
    result = df[df["tripId"] == tripID].sort_values(by=['vehicleTimestamp', 'vehicleStopSequence'])
    return result[["tripId","vehicleTimestamp","vehicleCurrentStatus", "vehicleStopSequence", "vehicleStopID"]]

In [44]:
def get_timings(df, tripID, expectedNumSequence):
    one_trip_df = viewOneTrip(df,tripID).copy()
    one_trip_df.reset_index(drop=True,inplace=True)
    one_trip_df2 = one_trip_df.groupby("vehicleStopSequence")

    counter = 1 # Starting sequence will be 1
    timings = [] # Output
    curStopArr = 0
    prevStopDep = 0
    
    for stopSequence, group in one_trip_df2:
        # This level deals with each stopSequece group as a whole. [Eg. All sequences from 1 to expectedNumSequence]

        # Ignore if 0. There were some off cases where stopSequence = 0. Ignore
        if stopSequence == 0: 
            continue
        
        # Add placeholder timestamps for missing sequences until counter matches the stopSequence. 
        # [Eg. Data only has stop sequence 20 to 30. This WHILE loop adds in [0,0] from 1 to 19]
        while counter < stopSequence:
            timings.append([0, 0])
            counter += 1

        for row_index, row in group.iterrows():
            # This level deals with a individual sequenceGroup's entries. [Eg. All entries within stopSequence = 5]

            # Stiutation 1: Only have one entry [Eg. stopSequence = 5 only have 1 entry]
            if len(group) == 1:
                # Status = IN_TRANSIT_TO, prevStopDep not set. Take the current StopSequence timestamp as the departure time of the previous stopSequence.
                # [Eg. stopSequence = 5 timestamp is stopSequence 4 departure]
                if row["vehicleStopSequence"] == counter and row["vehicleCurrentStatus"] == "IN_TRANSIT_TO" and prevStopDep == 0:
                    prevStopDep = row["vehicleTimestamp"]
                    timings[stopSequence-2][1] = prevStopDep

                # Status = STOPPED_AT, curStopArr not set. Take the current StopSequence timestamp as its arrival time.
                if row["vehicleCurrentStatus"] == "STOPPED_AT" and curStopArr == 0:
                    curStopArr = row["vehicleTimestamp"]

            # Situation 2: Multiple entries [Eg. stopSequence = 5 have 10 entries]
            else: 
                # Status = IN_TRANSIT_TO, prevStopDep not set. Continue is used here to take the current StopSequence first entry's timestamp as the departure time of the previous stopSequence.
                # [Eg. stopSequence = 5 have 10 entries, take first timestamp as stopSequence = 4 departure]
                if row["vehicleStopSequence"] == counter and row["vehicleCurrentStatus"] == "IN_TRANSIT_TO" and prevStopDep == 0:
                    prevStopDep = row["vehicleTimestamp"]
                    timings[stopSequence-2][1] = prevStopDep
                    continue

                # Get last transit as pass by timing
                if row["vehicleStopSequence"] == counter and row["vehicleCurrentStatus"] == "IN_TRANSIT_TO" and curStopArr == 0:
                    curStopArr = 0

                # Status = STOPPED_AT, curStopArr not set. Break is used here to take the current StopSequence first entry's timestamp as its arrival.
                # [Eg. stopSequence = 5 have 10 entries, take first timestamp as its arrival]
                elif row["vehicleCurrentStatus"] == "STOPPED_AT" and curStopArr == 0:
                    curStopArr = row["vehicleTimestamp"]
                    break
        
        # Reset the curStopArr & prevStopDep, increment the counter, append curStopArr for the current stopSequence.
        # [Eg. Append the curStopArr and departure = 0 for stopSequence = 5]
        if stopSequence <= expectedNumSequence:
            timings.append([curStopArr,0])
            curStopArr = 0
            prevStopDep = 0
            counter += 1

    # Add placeholder timings for missing stopSequence until it hits expectedNumSequence
    while len(timings) < expectedNumSequence:
        timings.append([0, 0])

    # Return the timings
    return timings

In [45]:
def getServicePollDate(file_path):

    file_name = re.search(r'[^/\\]+$', file_path).group()

    all_info = file_name.split("_")
    service = all_info[2]
    poll_rate = all_info[3]
    date = datetime.strptime(all_info[4].split(".")[0], '%d-%m-%Y')

    return [service, poll_rate, date]

def getTripTimingsByDirection(directedTripIds, expectedNumSequence):
    timingArray = []
    for tripId in directedTripIds:
        timingArray.append(get_timings(tripId, expectedNumSequence))
    return timingArray

def distinguish_one_trip_data(df, tripID):
    result = df[df["tripId"] == tripID].sort_values(by=['vehicleTimestamp', 'vehicleStopSequence'])
    return result[["tripId","vehicleTimestamp","vehicleCurrentStatus", "vehicleStopSequence", "vehicleStopID"]]

In [46]:
# Read Stops.txt and csv
stopsTxt = pd.read_csv("stops.txt", delimiter='\t')
file_path = "./raw_data/gtfs_data_2_15s_09-10-2023.csv"
# file_path = "gtfs_data_2_15s_07-10-2023.csv"


orig_df = pd.read_csv(file_path)
orig_df["vehicleTimestamp"] = pd.to_datetime(orig_df["vehicleTimestamp"], unit="s")


service_poll_date = getServicePollDate(file_path)
service = service_poll_date[0]
poll_rate = service_poll_date[1]
date = service_poll_date[2]

print("Service is:", service)
print("Poll rate is:", poll_rate)
print("Date is:", date)

Service is: 2
Poll rate is: 15s
Date is: 2023-10-09 00:00:00


In [47]:
# Cleaning & Transforming #
orig_df.drop(["pippenId", "pippenCreatedAt", "pippenPollingRate", "tripStartTime", "tripStartDate", "tripRouteId"], axis=1,inplace = True)
orig_df["tripDirectionId"] = np.where((orig_df["vehicleLabel"] == "FX2 To Portland") | (orig_df["vehicleLabel"] == "FX2 To NW 5th & Hoyt") | (orig_df["vehicleLabel"] == "FX2 To NW Irving & 5th"), 1, 0)
orig_df.drop_duplicates(inplace=True)
orig_df = orig_df[orig_df.vehicleStopSequence != 0]
orig_df.sort_values(by=['tripId', 'vehicleTimestamp'], inplace=True)

# Create a new DataFrame to store the sorted results
sorted_df = pd.DataFrame(columns=orig_df.columns)

for trip_id, group in orig_df.groupby('tripId'):
    sorted_group = group.sort_values(by='vehicleTimestamp', ascending=True)
    sorted_df = pd.concat([sorted_df, sorted_group], ignore_index=True)

# Reset the index of the sorted DataFrame
sorted_df.reset_index(drop=True, inplace=True)

/var/folders/sm/795yv_kj01z2spgzwlq0syth0000gn/T/ipykernel_31548/1549847525.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sorted_df = pd.concat([sorted_df, sorted_group], ignore_index=True)


In [48]:
# full_seq_dfs = orig_df.groupby('tripId').filter(lambda x: ((x.vehicleStopSequence.max() >= 42) & (x.vehicleStopSequence.min() <= 2)))
full_seq_dfs = sorted_df.groupby('tripId').filter(lambda x: ((x.vehicleStopSequence.max() >= 42) & (x.vehicleStopSequence.min() <= 2)))
print("Total number of records in both (0 & 1) directions:",len(full_seq_dfs))
print("Total number of trips in both (0 & 1) directions:",len(full_seq_dfs["tripId"].unique()))

Total number of records in both (0 & 1) directions: 29878
Total number of trips in both (0 & 1) directions: 171


In [49]:
zeroTrips = full_seq_dfs[full_seq_dfs["tripDirectionId"] == 0].reset_index(drop=True)
oneTrips = full_seq_dfs[full_seq_dfs["tripDirectionId"] == 1].reset_index(drop=True)

zeroTripIDs = zeroTrips["tripId"].unique()
oneTripIDs = oneTrips["tripId"].unique()

print("Number of Trips in 0 direction:",len(zeroTripIDs))
print("Number of Trips in 1 direction:",len(oneTripIDs))

Number of Trips in 0 direction: 85
Number of Trips in 1 direction: 86


# Split data into 4 time periods

In [50]:
def getUniqueTripIDs(subdf):
    return subdf["tripId"].unique()

# Get Timings and Impute the missing Timings

In [51]:
def fix_timing(timing):
    for i in range(len(timing)):
        departure = timing[i][1]
        if departure == 0:
            last_known_departure = timing[i - 1][1] if i > 0 else None
            next_known_arrival = None
            for j in range(i + 1, len(timing)):
                next_arr = timing[j][0]
                next_dep = timing[j][1]
                if next_arr != 0:
                    next_known_arrival = next_arr
                    break
                elif next_arr == 0 and next_dep != 0:
                    next_known_arrival = next_dep
                    break

            # Fix for missing first stop departure
            if i == 0:
                if timing[i+1][0] != 0:
                    timing[i][1] = timing[i+1][0] - timedelta(seconds = 30)
                elif timing[i+1][1] != 0: 
                    timing[i][1] = timing[i+1][1] - timedelta(seconds = 30)
                else:
                    timing[i][1] = timing[i][0] + timedelta(seconds = 30)

            # Fix for missing last stop arrival
            if i == len(timing)-1:
                if timing[i-1][1] != 0:
                    timing[i][0] = timing[i-1][1] + timedelta(seconds = 30)

            
            if last_known_departure is not None and next_known_arrival is not None:
                num_missing_stops = j - i
                time_difference = (next_known_arrival - last_known_departure).total_seconds()
                time_interval = timedelta(seconds=time_difference / (num_missing_stops + 1))

                for k in range(i, j):
                    if timing[k][1] == 0:
                        last_known_departure += time_interval
                        if timing[k][0]!= 0 and timing[k][0] > last_known_departure:
                            difference = (timing[k][0] - last_known_departure).total_seconds()
                            timing[k][1] = last_known_departure.replace(microsecond=0) + timedelta(seconds=difference)
                        else:
                            timing[k][1] = last_known_departure.replace(microsecond=0)

In [52]:
def fix_direction_timings(all_trip_timings):
    for each_trip_timings in all_trip_timings:
        fix_timing(each_trip_timings)

In [53]:
def direction_trip_timings(directedTrips, num_stops):
    result = []
    directedTripIds = getUniqueTripIDs(directedTrips)
    for i in range(len(directedTripIds)):
        result.append(get_timings(directedTrips, directedTripIds[i], num_stops))

    return result

In [54]:
zeroTimings = direction_trip_timings(zeroTrips,42)
oneTimings = direction_trip_timings(oneTrips,42)

In [ ]:
fix_direction_timings(zeroTimings)
fix_direction_timings(oneTimings)

# Get Interstation

## Each trip

In [ ]:
def get_interstation(tripsTimings):
    result = []
    
    for i in range(len(tripsTimings)-1):
        stopAArr = tripsTimings[i][0]
        stopADep = tripsTimings[i][1]
        stopBArr = tripsTimings[i+1][0]
        stopBDep = tripsTimings[i+1][1]
        
        if stopBArr == 0 and stopBDep != 0:
            stopBArr = stopBDep

        if stopADep == 0 and stopAArr != 0:
            stopADep = stopAArr + timedelta(seconds=30)

        if stopADep == 0 and stopAArr == 0:
            stopADep = stopBArr - timedelta(seconds=30)

        if stopBArr == 0 and stopBDep == 0:
            stopBArr = stopADep + timedelta(seconds=30)
        
        interstation = stopBArr - stopADep

        result.append(int(interstation.total_seconds()))
    return result

## All trips

In [ ]:
def direction_trips_interstation(directedTrips):
    result = []
    for each_trip in directedTrips:
        each_trip_interstation = get_interstation(each_trip)
        result.append(each_trip_interstation)
    return result


In [ ]:
zero_interstation = direction_trips_interstation(zeroTimings)
one_interstation = direction_trips_interstation(oneTimings)

# Get Arrivals

In [ ]:
def get_arrivals(tripsTimings,date): # Get Arrival Times [Each stop, per trip]
    result = []

    for i in range(len(tripsTimings)): # Create as many empty list as there is in tripsTimings. Allows for easy append via index.
        result.append([])

    for j in range(len(tripsTimings)): # For each trip
        for stop in tripsTimings[j]: # For each stop 
            stopArr = stop[0]
            stopDep = stop[1]

            if stopArr != 0: # Arrival time is present and is a datetime object.
                result[j].append(int((stopArr - date).total_seconds()))

            elif stopArr == 0 and stopDep != 0: # No arrival time, but have departure time and is a datetime object. Treat as bus pass by the stop.
                result[j].append(int((stopDep - date).total_seconds()))

            else:
                result[j].append(0)

    return result

# For Output

In [ ]:
def generate_arrival_rate(directedDF, num_stops, input_key_stops): 
    arrival_rate_list = []

    stops_in_route = directedDF["vehicleStopID"].unique()
    for i in range(num_stops-1):
        arr = 0

        if stops_in_route[i] in input_key_stops:
            arr += 0.05

        if i <(num_stops//2):
            arr += round(random.uniform(0.005, 0.03),3)
        else:
            arr += round(random.uniform(0.005, 0.049),3)

        arrival_rate_list.append(arr)

    arrival_rate_list.append(0)
    return arrival_rate_list

In [ ]:
def generate_alighting_percentage(directedDF, num_stops, input_key_stops):
    alighting_percentage_list = []

    stops_in_route = directedDF["vehicleStopID"].unique()

    alighting_percentage_list.append(0)

    for i in range(num_stops-3):
        a = 0

        if stops_in_route[i] in input_key_stops:
            a += 0.02

        if i <(num_stops//2):
            a += round(random.uniform(0, 0.6),3)
        else:
            a += round(random.uniform(0, 0.97),3)
            
        alighting_percentage_list.append(a)

    alighting_percentage_list.append(1)

    return alighting_percentage_list

In [ ]:
def generate_initial_passengers(directedDF, num_stops, input_key_stops):
    initial_passengers_list = []

    stops_in_route = directedDF["vehicleStopID"].unique()
    for i in range(num_stops-1):
        p = 0

        if stops_in_route[i] in input_key_stops:
            p += 4

        if i <(num_stops//2):
            p += random.randint(0, 6)
        else:
            p += random.randint(0, 4)
            
        initial_passengers_list.append(p)

    initial_passengers_list.append(0)
    return initial_passengers_list

In [ ]:
def generate_weights(directedDF, num_stops, input_key_stops): 
    stop_sequence = 1
    row = 0

    weights_list = [0.5] * num_stops
    for i in range(num_stops):
        green = 1
        while stop_sequence == int(directedDF["vehicleStopSequence"][row]):
            if int(directedDF["vehicleStopID"][row]) in input_key_stops and green:
                weights_list[stop_sequence-1] += 0.2
                green = 0
            if weights_list[stop_sequence-1] <=1 and directedDF["vehicleCurrentStatus"][row] == "STOPPED_AT":
                weights_list[stop_sequence-1] +=0.05
            row+=1
        stop_sequence +=1

    return weights_list

In [ ]:
def get_stop_info(directedDF, num_stops ,staticStopsFile, actualStopNameSeq, stops_to_ignore):
    stopsTxt = pd.read_csv(staticStopsFile, delimiter='\t')

    bus_stops = directedDF["vehicleStopID"].unique().astype(int).astype(str)

    rand_coordinates_list = []
    rand_stop_ids_list = []
    rand_stop_names_list = []

    for i in range(num_stops):
        rand_coordinates_list.append([])

    # Randomly Take
    for idx, each_stop in enumerate(bus_stops):
        for i in range(len(stopsTxt)):
            row = stopsTxt["stop_id,stop_code,stop_name,tts_stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,direction,position"][i]
            one_stop = row.split(",")
            stop_id = str(one_stop[0])
            stop_name = str(one_stop[2])

            if stop_id in bus_stops and stop_id not in stops_to_ignore and each_stop == stop_id:
                try:
                    stop_location_lat = float(one_stop[5])
                    stop_location_long = float(one_stop[6])
                except ValueError:
                    stop_location_lat = float(one_stop[6])
                    stop_location_long = float(one_stop[7])                                     
                
                rand_coordinates_list[idx].append(stop_location_lat)
                rand_coordinates_list[idx].append(stop_location_long)
                rand_stop_ids_list.append(stop_id)
                rand_stop_names_list.append(stop_name)

    # Fix Randomness 
    coordinates_list = []
    stop_ids_list = []
    stop_names_list = []

    for correctStopName in actualStopNameSeq:
        for i in range(len(rand_stop_names_list)):
            if correctStopName == rand_stop_names_list[i]:
                coordinates_list.append(rand_coordinates_list[i])
                stop_ids_list.append(rand_stop_ids_list[i])
                stop_names_list.append(rand_stop_names_list[i])
    
    return coordinates_list, stop_ids_list, stop_names_list

In [ ]:
def get_prev_dwell(firstTripTimings):
    result = []
    
    for i in range(len(firstTripTimings)-1): # For each stop in a trip
        stopArr = firstTripTimings[i][0]
        stopDep = firstTripTimings[i][1]

        # No arrival and departure time
        if stopArr == 0 and stopDep == 0:
            result.append(0)

        # No arrival time, but have departure time and is a datetime object. Bus passed by = No dwell time
        if stopArr == 0 and stopDep != 0:
            result.append(0)

        # Have both arrival and departure time and is a datetime object.
        if stopArr != 0 and stopDep != 0:
            dwell = stopDep - stopArr
            result.append(int(dwell.total_seconds()))

    return result

In [ ]:
def get_prev_arrival(firstTripTimings):
    result = []
    for each_stop in firstTripTimings:
        stopArr = each_stop[0]
        stopDep = each_stop[1]

        if stopArr != 0: # Arrival time is present and is a datetime object.
            result.append(int((stopArr - date).total_seconds()))

        elif stopArr == 0 and stopDep != 0: # No arrival time, but have departure time and is a datetime object. Treat as bus pass by the stop.
            result.append(int((stopDep - date).total_seconds()))

        else:
            result.append(0)

    return result

In [ ]:
def generate_target_headway(num_trips, num_stops, target_headway):
    result = []
    for i in range(num_trips):
        result.append([])
        for j in range(num_stops):
            result[i].append(target_headway)
    return result



In [ ]:
def get_bus_availability(directed_timings, date):
    result = []

    for i in range(len(directed_timings)): # For each trip
        lastRecordIndex = len(directed_timings[i])-1
        lastRecord = directed_timings[i][lastRecordIndex]
        while True:
            if lastRecord[1] != 0: # Last record departure is a datetime object.
                result.append(int((lastRecord[1] - date).total_seconds()))
                break
            elif lastRecord[0] != 0: # Last record departure is a datetime object. Missing departure
                result.append(int((lastRecord[0] - date).total_seconds()))
                break

            lastRecordIndex = lastRecordIndex-1
            lastRecord = directed_timings[i][lastRecordIndex]

    return result

In [ ]:
def createOriginalDispatch(directedTimings, date):
    dispatchList = []
    result = []
    for i in range(len(directedTimings)):
        dispatch = directedTimings[i][0][1] + timedelta(seconds=random.randint(-120, 120))
        dispatchList.append(dispatch)

    for eachDispatch in dispatchList:
        result.append(int((eachDispatch - date).total_seconds()))
    return result

In [ ]:
def createBusAvailabilities(num_trips, date):
    result = [0] * num_trips
    return result 

## Common Values

In [ ]:
bus_capacity = 100
boarding_duration = 2
alighting_duration = 2
max_allowed_deviation = 600
target_headway = 800 # All 82 trips including Trip 0 takes 65000s, or about 800s between each bus
penalty_coefficient = 10000

## For Zero Bound Trips

In [ ]:
zeroStopsSequence = pd.read_csv("./Scraper/zeroStops.csv")
zeroStopsNames = zeroStopsSequence["0"].unique()
zero_num_stops = len(zeroStopsNames)

zero_stops_to_ignore = ["0", "3007"]
zeroStopsInfo = get_stop_info(zeroTrips, zero_num_stops, "stops.txt", zeroStopsNames, zero_stops_to_ignore)

In [ ]:
zero_num_trips = len(zeroTimings) - 4
zero_original_dispatch_list = createOriginalDispatch(zeroTimings[4:],date)
zero_prev_arrival_list = get_prev_arrival(zeroTimings[3])
zero_prev_dwell_list = get_prev_dwell(zeroTimings[3])
zero_bus_availability_list = createBusAvailabilities(zero_num_trips, date)
zero_target_headway_2dlist = generate_target_headway(zero_num_trips, zero_num_stops, target_headway)
zero_interstation_travel_2dlist = zero_interstation[4:]


zero_input_key_stops = [9302, 7634, 1458, 1497, 1379, 1415, 8199, 14230]
zero_weights_list = generate_weights(zeroTrips, zero_num_stops, zero_input_key_stops) 
zero_arrival_rate_list = generate_arrival_rate(zeroTrips, zero_num_stops, zero_input_key_stops)
zero_initial_passengers_list = generate_initial_passengers(zeroTrips, zero_num_stops, zero_input_key_stops)
zero_initial_passengers_list = generate_initial_passengers(zeroTrips, zero_num_stops, zero_input_key_stops)
zero_alighting_percentage_list = generate_alighting_percentage(zeroTrips, zero_num_stops, zero_input_key_stops)

zero_coordinates_list = zeroStopsInfo[0]
zero_stop_ids_list = zeroStopsInfo[1]
zero_stop_names_list = zeroStopsInfo[2]


## For One Bound Trips

In [ ]:
# len(oneTrips["tripId"].unique())

In [ ]:
# oneTrips.head()

In [ ]:
# oneTrips.tail()

In [ ]:
# oneStopsSequence = pd.read_csv("oneStops.csv")
# oneStopsNames = oneStopsSequence["1"].unique()
# one_num_stops = len(oneStopsNames) - 1

# one_stops_to_ignore = ["9302", "3007"]
# oneStopsInfo = get_stop_info(oneTrips, one_num_stops, "stops.txt",oneStopsNames, one_stops_to_ignore)

In [ ]:
# for i in range(len(zeroTimings)):
#     print(10*"#","Trip number",i+1,10*"#")
#     for j in range(len(zeroTimings[i])):
#         arr = zeroTimings[i][j][0]
#         dep = zeroTimings[i][j][1]
#         print(j+1, "[Arr]:",arr,"[Dep]",dep)
#     print("\n")

In [ ]:
# one_num_trips = len(oneTimings) - 1
# one_original_dispatch_list = createOriginalDispatch(oneTimings, date)
# one_prev_arrival_list = get_prev_arrival(oneTimings[0])
# one_prev_dwell_list = get_prev_dwell(oneTimings[0])
# one_bus_availability_list = get_bus_availability(zeroTimings, date)
# one_target_headway_2dlist = generate_target_headway(one_num_trips, one_num_stops, target_headway)
# one_interstation_travel_2dlist = one_interstation[1:]


# one_input_key_stops = [14232, 14233, 1416, 1381, 1499, 1459, 7800, 9302]
# one_weights_list = generate_weights(oneTrips, one_num_stops, one_input_key_stops) 
# one_arrival_rate_list = generate_arrival_rate(oneTrips, one_num_stops, one_input_key_stops)
# one_initial_passengers_list = generate_initial_passengers(oneTrips, one_num_stops, one_input_key_stops)
# one_alighting_percentage_list = generate_alighting_percentage(oneTrips, one_num_stops, one_input_key_stops)  

# one_coordinates_list = oneStopsInfo[0]
# one_stop_ids_list = oneStopsInfo[1]
# one_stop_names_list = oneStopsInfo[2]


In [ ]:
def jsonOutput(num_trips, num_stops, bus_capacity, original_dispatch_list, coordinates_list, stop_ids_list, stop_names_list, prev_arrival_list, prev_dwell_list, arrival_rate_list, alighting_percentage_list, boarding_duration, alighting_duration, weights_list, bus_availability_list, initial_passengers_list, max_allowed_deviation, target_headway_2dlist, interstation_travel_2dlist):
    output = {
        "num_trips": num_trips,
        "num_stops": num_stops,
        "bus_capacity": bus_capacity,
        "original_dispatch_list": original_dispatch_list,
        "coordinates_list": coordinates_list,
        "stop_ids_list": stop_ids_list,
        "stop_names_list": stop_names_list,
        "prev_arrival_list": prev_arrival_list,
        "prev_dwell_list": prev_dwell_list,
        "arrival_rate_list": arrival_rate_list,
        "alighting_percentage_list": alighting_percentage_list,
        "boarding_duration": boarding_duration,
        "alighting_duration": alighting_duration,
        "weights_list": weights_list, 
        "bus_availability_list": bus_availability_list,
        "initial_passengers_list": initial_passengers_list,
        "max_allowed_deviation": max_allowed_deviation,
        "penalty_coefficient": penalty_coefficient,
        "target_headway_2dlist": target_headway_2dlist,
        "interstation_travel_2dlist": interstation_travel_2dlist 
    }

    # Serializing json
    json_object = json.dumps(output, indent=4)
    
    # Writing to input.json
    with open("input.json", "w") as outfile:
        outfile.write(json_object)


In [ ]:

jsonOutput(zero_num_trips, zero_num_stops, bus_capacity, zero_original_dispatch_list, zero_coordinates_list, zero_stop_ids_list, zero_stop_names_list, zero_prev_arrival_list, zero_prev_dwell_list, zero_arrival_rate_list, zero_alighting_percentage_list, boarding_duration, alighting_duration, zero_weights_list, zero_bus_availability_list, zero_initial_passengers_list, max_allowed_deviation, zero_target_headway_2dlist, zero_interstation_travel_2dlist)